In [86]:
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from keras.utils import load_img, img_to_array, to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model, load_model
import numpy as np

In [3]:
images_base_path = r'/Users/praneethkumarpalepu/Downloads/animals/images'

In [42]:
def image_dataset_loader(base_path, image_size=(224, 224), test_size=0.2, random_state=0, batch_size= 64):
    images, labels = [], []
    image_file_location, act_label = [], []
    for folder in os.listdir(base_path):
        if not folder.startswith('.'):
            folder_path = os.path.join(base_path, folder)
            for file in os.listdir(folder_path):
                _, ext = os.path.splitext(file)
                if ext.lower() in ['.jpg', '.jpeg', '.png']:
                    file_path = os.path.join(folder_path, file)
                    image_file_location.append(file_path)
                    act_label.append(folder)

    total_files = len(image_file_location)
    quotient = total_files//batch_size
    reminder = total_files % batch_size
    total_batches = quotient+1 if reminder > 0 else quotient

    for batch in tqdm(range(total_batches)):
        start_index = batch * batch_size
        end_index = (batch * batch_size)+batch_size
        if end_index > total_files:
            end_index = end_index - total_files
        image_batch = image_file_location[start_index:end_index]
        label = act_label[start_index:end_index]
        counter= 0
        tmp_image, tmp_labels = [], []
        for file in image_batch:           
            image = load_img(file, target_size=image_size)
            image_arr = img_to_array(image)
            tmp_image.append(image_arr)
            tmp_labels.append(label[counter])
            counter+=1

        images.extend(np.array(tmp_image))
        labels.extend(np.array(tmp_labels))

    images = np.array(images)
    labels = np.array(labels)

    xtr, xte, ytr, yte = train_test_split(images, labels, test_size= test_size, random_state= random_state)

    return (xtr, ytr), (xte, yte)

In [46]:
(xtr, ytr), (xte, yte) = image_dataset_loader(images_base_path, batch_size=128)

100%|███████████████████████████████████████████| 24/24 [00:04<00:00,  5.89it/s]


In [47]:
xtr.shape

(2355, 224, 224, 3)

In [48]:
xte.shape

(589, 224, 224, 3)

In [50]:
encoder = LabelEncoder()

In [51]:
ytr = encoder.fit_transform(ytr)

In [52]:
yte = encoder.transform(yte)

In [53]:
ytr = to_categorical(ytr)

In [54]:
yte = to_categorical(yte)

In [55]:
xtr = xtr.astype('float32')/255
xte = xte.astype('float32')/255

In [59]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

In [60]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
history = model.fit(xtr, ytr, validation_data=(xte, yte), epochs=2, batch_size=256)

Epoch 1/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 59s 6s/step - accuracy: 0.9259 - loss: 0.1945 - val_accuracy: 0.7334 - val_loss: 0.8837
Epoch 2/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 57s 6s/step - accuracy: 0.9581 - loss: 0.1172 - val_accuracy: 0.7453 - val_loss: 0.9729


In [62]:
model.save('animals_cnn.keras')

In [67]:
new_model = load_model(r'animals_cnn.keras')

In [71]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_13 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 52, 52, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 10, 10, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 5, 5, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 3, 3, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1024)           │       132,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,251,339 (12.40 MB)

 Trainable params: 1,083,779 (4.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,167,560 (8.27 MB)

In [88]:
interm_model = Model(inputs = model.layers[0].input, outputs = model.layers[-4].output)

In [89]:
interm_model.summary()

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 52, 52, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 10, 10, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 5, 5, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 3, 3, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 128)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 425,344 (1.62 MB)

 Trainable params: 425,344 (1.62 MB)

 Non-trainable params: 0 (0.00 B)

In [90]:
xtr_features = interm_model.predict(xtr)

74/74 ━━━━━━━━━━━━━━━━━━━━ 11s 141ms/step


In [91]:
xtr_features.shape

(2355, 128)

In [92]:
xte_features = interm_model.predict(xte)

19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 129ms/step


In [93]:
rfc = RandomForestClassifier(n_estimators=250, random_state=0)

In [94]:
rfc.fit(xtr_features, ytr)

RandomForestClassifier(n_estimators=250, random_state=0)

In [95]:
rfc_pred = rfc.predict(xte_features)

In [96]:
accuracy_score(yte, rfc_pred)

0.7385398981324278